# 🔬 Deepfake Detection Research - Google Colab

## Konfiguracja GPU:
**Runtime → Change runtime type → GPU**

| GPU | VRAM | Rekomendacja |
|-----|------|-------------|
| H100 | 80GB | 🏆 Najlepsza (Pro+) |
| A100 | 40GB | ⭐ Świetna (Pro) |
| L4 | 24GB | ✅ Dobra (Pro) |
| **T4** | 16GB | ✅ **Free - wybierz to!** |

**Włącz też: Duża ilość pamięci RAM** ✅

## 1️⃣ Sprawdź GPU

In [ ]:
# Sprawdź GPU
!nvidia-smi

import torch
print(f"\n{'='*50}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    vram = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"GPU: {gpu_name}")
    print(f"VRAM: {vram:.1f} GB")
    
    # Rekomendacja batch size
    if vram >= 40:
        print("\n🎯 Rekomendowany batch size: 64-128")
    elif vram >= 20:
        print("\n🎯 Rekomendowany batch size: 32-64")
    elif vram >= 15:
        print("\n🎯 Rekomendowany batch size: 16-32")
    else:
        print("\n🎯 Rekomendowany batch size: 8-16")

## 2️⃣ Sklonuj Repozytorium

In [ ]:
# Repozytorium GitHub
GITHUB_REPO = "kordin33/DeepFake"

# Sklonuj repo
!git clone https://github.com/{GITHUB_REPO}.git

# Wejdź do folderu
%cd DeepFake

# Pokaż strukturę
!ls -la

## 3️⃣ Zainstaluj Zależności

In [ ]:
# Instaluj dodatkowe pakiety
!pip install -q scikit-learn tqdm matplotlib seaborn opencv-python-headless
!pip install -q wandb  # Weights & Biases
!pip install -q datasets huggingface_hub  # Do pobierania danych

print("\n✅ Zależności zainstalowane!")

## 4️⃣ 🔐 Zaloguj się do Weights & Biases

In [ ]:
import wandb

# Zaloguj się do W&B (pojawi się link do wpisania klucza API)
wandb.login()

print("\n✅ Zalogowano do Weights & Biases!")

## 5️⃣ Konfiguracja Eksperymentu

In [ ]:
# =============================================================================
# KONFIGURACJA - ZMIEŃ TUTAJ
# =============================================================================

# W&B Project
WANDB_PROJECT = "msc-deepfake-detection"

# Eksperymenty do uruchomienia
EXPERIMENT = "all"  # "all", "baseline", "advanced", "ultimate"

# Training
EPOCHS = 20
BATCH_SIZE = 32  # Dostosuj do GPU (T4: 16-32, A100: 64-128)
LEARNING_RATE = 1e-4
IMG_SIZE = 224

# Data
MAX_PER_CLASS_A = 5000  # Max samples per class dla datasetu A
MAX_PER_CLASS_B = 2000  # Max samples per class dla datasetu B

# Augmentation
USE_SBI = True  # Self-Blended Images (rekomendowane!)

# CUDA
USE_COMPILE = True  # torch.compile() - 20-40% speedup

# Seed
SEED = 42

print(f"📊 Konfiguracja:")
print(f"   Experiment: {EXPERIMENT}")
print(f"   Epochs: {EPOCHS}")
print(f"   Batch size: {BATCH_SIZE}")
print(f"   Use SBI: {USE_SBI}")
print(f"   Use compile: {USE_COMPILE}")

## 6️⃣ Pobierz Dane z HuggingFace

In [ ]:
# Pobierz i przygotuj dane używając wbudowanego skryptu
!python efficientnet_b0_deepfake.py --prepare --data-root ./data \
    --max-per-class-a {MAX_PER_CLASS_A} \
    --max-per-class-b {MAX_PER_CLASS_B}

# Sprawdź dane
print("\n📁 Struktura danych:")
!find ./data -type d | head -20

In [ ]:
# Policz pliki w każdym folderze
import os
from pathlib import Path

data_root = Path("./data")
for split_dir in data_root.rglob("*"):
    if split_dir.is_dir() and (split_dir / "fake").exists():
        fake_count = len(list((split_dir / "fake").glob("*")))
        real_count = len(list((split_dir / "real").glob("*")))
        print(f"{split_dir.relative_to(data_root)}: fake={fake_count}, real={real_count}")

## 7️⃣ Quick Test

In [ ]:
# Test czy wszystkie moduły działają
!python quick_test.py

## 8️⃣ Setup CUDA Optimizations

In [ ]:
import sys
sys.path.insert(0, '.')

from deepfake_research.utils.cuda_utils import (
    setup_cuda_optimizations,
    print_cuda_memory_stats,
)

# Setup optimizations
cuda_config = setup_cuda_optimizations(
    use_compile=USE_COMPILE,
    use_cudnn_benchmark=True,
    use_tf32=True,
    verbose=True,
)

print("\n" + "="*50)
print_cuda_memory_stats()

## 9️⃣ 🚀 Uruchom Eksperymenty z W&B Logging

In [ ]:
# Uruchom eksperymenty z logowaniem do W&B
!python run_experiments.py \
    --experiment {EXPERIMENT} \
    --epochs {EPOCHS} \
    --batch-size {BATCH_SIZE} \
    --lr {LEARNING_RATE} \
    --data-root ./data \
    --output-dir ./experiments \
    --wandb \
    --wandb-project {WANDB_PROJECT} \
    {'--use-sbi' if USE_SBI else ''} \
    {'--compile' if USE_COMPILE else ''} \
    --seed {SEED}

## 🔟 Alternatywnie: Ręczne Trenowanie z Pełną Kontrolą

In [ ]:
import sys
sys.path.insert(0, '.')

import torch
import wandb
from deepfake_research.models.factory import create_model
from deepfake_research.data.datasets import create_dataloaders
from deepfake_research.training.trainer import Trainer
from deepfake_research.training.optimizers import get_optimizer, get_scheduler
from deepfake_research.training.losses import DeepfakeLoss
from deepfake_research.utils.cuda_utils import compile_model

# Lista eksperymentów do uruchomienia
MODELS_TO_TRAIN = [
    "baseline_efficientnet",
    "baseline_vit", 
    "freq_efficientnet",
    "attention_efficientnet",
    "hybrid",
    "ultimate",
]

# Lub pojedynczy model:
# MODELS_TO_TRAIN = ["ultimate"]

device = "cuda" if torch.cuda.is_available() else "cpu"

# Załaduj dane
print("Loading data...")
loaders = create_dataloaders(
    data_root="./data",
    batch_size=BATCH_SIZE,
    num_workers=2,
    use_sbi=USE_SBI,
    sbi_probability=0.3,
)

print(f"Train: {len(loaders['train'].dataset)} samples")
print(f"Val: {len(loaders['val'].dataset)} samples")
print(f"Test A: {len(loaders['test_A'].dataset)} samples")
print(f"Test B: {len(loaders['test_B'].dataset)} samples")

In [ ]:
# Trenuj każdy model z logowaniem do W&B
results = {}

for model_name in MODELS_TO_TRAIN:
    print(f"\n{'='*60}")
    print(f"🚀 Training: {model_name}")
    print(f"{'='*60}")
    
    # Inicjalizuj W&B run
    run = wandb.init(
        project=WANDB_PROJECT,
        name=model_name,
        config={
            "model": model_name,
            "stage": "training",
            "img_size": IMG_SIZE,
            "batch_size": BATCH_SIZE,
            "lr": LEARNING_RATE,
            "epochs": EPOCHS,
            "use_sbi": USE_SBI,
            "use_compile": USE_COMPILE,
            "seed": SEED,
        },
        reinit=True,  # Pozwala na wiele runów w jednej sesji
    )
    
    try:
        # Stwórz model
        model = create_model(model_name)
        
        # Kompiluj dla speedup
        if USE_COMPILE:
            model = compile_model(model, mode="default")
        
        # Optimizer
        optimizer = get_optimizer(model, lr=LEARNING_RATE)
        scheduler = get_scheduler(optimizer, epochs=EPOCHS)
        criterion = DeepfakeLoss(loss_type='smooth', label_smoothing=0.1)
        
        # Trainer
        trainer = Trainer(
            model=model,
            train_loader=loaders['train'],
            val_loader=loaders['val'],
            optimizer=optimizer,
            scheduler=scheduler,
            criterion=criterion,
            device=device,
            epochs=EPOCHS,
            use_amp=True,
            early_stopping=True,
            patience=5,
            save_dir=f"./experiments/{model_name}",
            experiment_name=model_name,
            use_wandb=True,
        )
        
        # Trenuj
        result = trainer.train()
        results[model_name] = result
        
        print(f"\n✅ {model_name}: Best Val Acc = {result['best_val_acc']:.4f}")
        
    except Exception as e:
        print(f"\n❌ Error training {model_name}: {e}")
        
    finally:
        wandb.finish()

print("\n" + "="*60)
print("🏁 All training completed!")
print("="*60)

## 1️⃣1️⃣ Benchmark i Porównanie

In [ ]:
from deepfake_research.evaluation.benchmark import Benchmark
from deepfake_research.evaluation.metrics import MetricsComputer

# Wczytaj wytrenowane modele i zbenchmarkuj
print("Loading trained models for benchmarking...")

# Inicjalizuj W&B dla benchmarku
benchmark_run = wandb.init(
    project=WANDB_PROJECT,
    name="benchmark_comparison",
    config={"stage": "benchmark"},
    reinit=True,
)

test_loaders = {
    'test_A': loaders['test_A'],
    'test_B': loaders['test_B'],
}

benchmark = Benchmark(
    dataloaders=test_loaders,
    device=device,
    output_dir="./experiments/benchmark",
)

# Dodaj każdy wytrenowany model do benchmarku
from pathlib import Path

for model_name in MODELS_TO_TRAIN:
    checkpoint_path = Path(f"./experiments/{model_name}/{model_name}_best.pth")
    if checkpoint_path.exists():
        print(f"\nBenchmarking {model_name}...")
        model = create_model(model_name)
        checkpoint = torch.load(checkpoint_path, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        model = model.to(device)
        
        benchmark.add_model(model, model_name)

# Print and log comparison
benchmark.print_comparison()

# Log to W&B
comparison = benchmark.compare()
for model_name, data in comparison['models'].items():
    wandb.log({
        f"{model_name}/mean_accuracy": data['mean_accuracy'],
        f"{model_name}/mean_auc": data['mean_auc'],
        f"{model_name}/params_millions": data['params_millions'],
    })

wandb.finish()

# Save results
benchmark.save_results("full_benchmark.json")

## 1️⃣2️⃣ Wyświetl Wyniki

In [ ]:
# Pokaż raport
!cat experiments/benchmark/BENCHMARK_REPORT.md

In [ ]:
# Wyświetl wykresy
from IPython.display import Image, display
import os

for img_file in ['cross_dataset_heatmap.png', 'model_comparison.png']:
    path = f'experiments/{img_file}'
    if os.path.exists(path):
        print(f"\n{img_file}:")
        display(Image(path))

In [ ]:
# Szczegółowe wyniki
import json

with open('experiments/benchmark/full_benchmark.json', 'r') as f:
    results = json.load(f)

print("=" * 60)
print("🏆 RANKING MODELI")
print("=" * 60)

# Sort by mean accuracy
sorted_models = sorted(
    results['detailed_results'].items(),
    key=lambda x: x[1]['summary']['mean_accuracy'],
    reverse=True
)

for i, (model_name, data) in enumerate(sorted_models, 1):
    summary = data['summary']
    medal = "🥇" if i == 1 else "🥈" if i == 2 else "🥉" if i == 3 else f"{i}."
    print(f"\n{medal} {model_name}")
    print(f"   Mean Accuracy: {summary['mean_accuracy']:.4f}")
    print(f"   Mean AUC: {summary['mean_auc']:.4f}")
    print(f"   Parameters: {summary['params_millions']:.2f}M")

## 1️⃣3️⃣ Zapisz na Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import shutil
from datetime import datetime

# Nazwa folderu z datą
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
dest_folder = f"/content/drive/MyDrive/deepfake_results_{timestamp}"

# Kopiuj wyniki
shutil.copytree("./experiments", dest_folder)
print(f"\n✅ Wyniki zapisane do: {dest_folder}")

# Pokaż co zapisano
!ls -la {dest_folder}

---

## 📊 Summary

### Links:
- **W&B Dashboard**: https://wandb.ai/YOUR_USERNAME/msc-deepfake-detection
- **GitHub Repo**: https://github.com/kordin33/DeepFake

### Next Steps:
1. Przejrzyj wyniki na W&B
2. Pobierz najlepszy model z checkpointu
3. Dodaj więcej danych lub eksperymentów
4. Przeprowadź ablation study